In [1]:
import pandas as pd
import import_data

ImportError: No module named import_data

In [1]:
data_path = '../data/'

In [4]:
import_data.parse('../data/events.json')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
